<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-03-20 10:02:57
-------------------
qualified stocks: 90
with latest results: 84
still star stocks: 46
-------------------
Initial Investment:  1.06 C
CY Investment:  1.19 C
Reserve:  4.00 K
Current:  1.09 C
-------------------
Today PnL: 42.47 K (0.39%)
Current PnL: -23.72 L (-19.91%)
CY Booked + Current PnL: -9.85 L (-8.26%)
-------------------
Total profit:  2.52 L
Total loss:  -26.24 L
-------------------
Total Booked + Current PnL: 3.26 L (3.08%)
Total Booked PnL: 26.98 L (25.44%)
Curr Year Booked PnL: 13.87 L (12.68%)
Prev Year Booked PnL: 13.11 L (12.36%)
Est FTT:  1.88 C
Est FTT PnL: 79.07 L (72.28%)
-------------------
Est LTT:  2.49 C
Est LTT PnL: 1.39 C (127.38%)
Deployed:  1.06 C
Current:  1.09 C
CAGR/XIRR %: 2.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NMDC,65.62,82.00,40.27,M-MC,67.78,104606.0,4207.0,20858.0,-0.60,4.19,19.94,24.96,507.0,0.20,0.96,13.95,XY25,NTT,MINING
12,AXISBANK,986.10,1317.30,-23.84,H-LC,82.22,107222.0,6640.0,27138.0,-0.08,6.60,25.31,33.59,NaN,0.24,0.98,10.87,X40,ATH,BANKS
47,LICI,750.57,983.00,-24.11,H-LC,55.56,103475.0,2899.0,28249.0,-0.59,2.88,27.30,30.97,202.0,0.10,0.95,5.90,XY25,NTT,INSURANCE
69,SBILIFE,1483.67,1924.99,-15.45,H-LC,70.00,101663.0,773.0,29238.0,0.68,0.77,28.76,29.75,NaN,0.03,0.93,11.88,AR,ATH,INSURANCE
70,SBIN,760.30,863.00,-24.12,M-LC,76.67,197571.0,-3908.0,31117.0,0.06,-1.94,15.75,13.51,93.0,-0.13,1.81,8.24,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,FINEORG,4771.46,7168.71,-31.16,H-SC,81.11,68285.0,-12830.0,53583.0,6.60,-15.82,78.47,50.24,-6.0,-0.24,0.62,14.66,X40N,ATH,CHEMICALS
22,COFFEEDAY,59.14,80.00,-123.94,L-SC,100.00,51821.0,-61728.0,101782.0,4.98,-54.36,196.41,35.27,NaN,-0.61,0.47,26.83,XR,NTT,HOTELS
11,AWL,369.21,485.00,-70.33,H-MC,41.11,121172.0,-54203.0,109200.0,3.97,-30.91,90.12,31.36,-36.0,-0.50,1.11,8.21,XY24,NTT,FMCG
46,LAOPALA,369.40,464.00,-27.98,H-SC,93.33,60652.0,-40194.0,66020.0,3.69,-39.86,108.85,25.61,-50.0,-0.61,0.55,9.39,AR,NTT,CERAMICS
84,TTKPRESTIG,769.29,1149.24,-12.42,X-SC,10.00,80067.0,-20710.0,70483.0,2.72,-20.55,88.03,49.39,1.0,-0.29,0.73,4.19,X40N,ATH,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HAVELLS,1588.50,2077.12,-5.12,X-LC,47.22,94261.0,-5814.0,36602.0,-3.96,-5.81,38.83,30.76,NaN,-0.16,0.86,6.33,X40,ATH,ELECTRICAL
20,CAMS,3643.00,5250.99,-6.84,H-SC,95.56,103586.0,1582.0,43444.0,-3.05,1.55,41.94,44.14,261.0,0.04,0.95,32.48,XR,ATH,MISC
78,TANLA,991.56,1987.55,-47.89,H-SC,66.67,112434.0,-106701.0,326812.0,-2.28,-48.69,290.67,100.45,-37.0,-0.33,1.03,20.29,AR,ATH,IT
3,ABB,5308.55,9007.77,-40.75,H-LC,62.22,96824.0,1270.0,65317.0,-1.76,1.33,67.46,69.68,NaN,0.02,0.89,8.99,AR,ATH,ELECTRICAL
27,EASEMYTRIP,18.68,26.40,-5.26,M-SC,88.89,115448.0,-50953.0,119720.0,-1.52,-30.62,103.70,41.33,-91.0,-0.43,1.06,17.39,XY24,NTT,TRAVEL


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NMDC,65.62,82.00,40.27,M-MC,67.78,104606.0,4207.0,20858.0,-0.60,4.19,19.94,24.96,507.0,0.20,0.96,13.95,XY25,NTT,MINING
29,GICRE,335.06,583.00,2.32,H-MC,92.22,180743.0,34322.0,74032.0,-1.16,23.44,40.96,74.00,109.0,0.46,1.65,32.18,XY24,BTT,INSURANCE
20,CAMS,3643.00,5250.99,-6.84,H-SC,95.56,103586.0,1582.0,43444.0,-3.05,1.55,41.94,44.14,261.0,0.04,0.95,32.48,XR,ATH,MISC
5,ADANIPORTS,1103.69,1583.00,-9.49,M-LC,87.78,159756.0,8550.0,57113.0,-0.63,5.65,35.75,43.43,NaN,0.15,1.46,10.74,XY24,NTT,MISC
67,SBICARD,765.91,1075.00,-9.82,H-MC,56.67,198352.0,18363.0,54269.0,-0.29,10.20,27.36,40.36,-1.0,0.34,1.81,27.53,XY24,NTT,FINANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,CAMS,3643.00,5250.99,-6.84,H-SC,95.56,103586.0,1582.0,43444.0,-3.05,1.55,41.94,44.14,261.0,0.04,0.95,32.48,XR,ATH,MISC
14,BANKINDIA,112.90,190.00,-35.22,M-MC,47.22,91047.0,-9208.0,77672.0,-0.66,-9.18,85.31,68.29,-12.0,-0.12,0.83,13.48,XR,NTT,BANKS
75,SONACOMS,578.05,1006.00,-32.02,M-MC,48.89,88996.0,-12163.0,87056.0,0.12,-12.02,97.82,74.03,-8.0,-0.14,0.81,7.24,AR,BTT,AUTO
1,AARTIIND,487.04,919.00,7.11,M-SC,26.67,66433.0,-14903.0,87041.0,-0.39,-18.32,131.02,88.69,-8.0,-0.17,0.61,5.80,XR,NTT,CHEMICALS
65,SAMMAANCAP,170.35,326.00,-157.18,M-SC,22.22,67968.0,-34242.0,127630.0,0.41,-33.50,187.78,91.37,NaN,-0.27,0.62,7.77,XR,NTT,FINANCE


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1022.60,1486.0,7.36,X-MC,73.33,187188.0,31753.0,38692.0,0.30,20.43,20.67,45.32,8.0,0.82,1.71,27.37,XY24,NTT,HEALTHCARE
67,SBICARD,765.91,1075.0,-9.82,H-MC,56.67,198352.0,18363.0,54269.0,-0.29,10.20,27.36,40.36,-1.0,0.34,1.81,27.53,XY24,NTT,FINANCE
2,AAVAS,1815.63,2328.0,-30.69,M-SC,98.89,208789.0,14517.0,40317.0,1.73,7.47,19.31,28.22,6.0,0.36,1.91,48.31,XY24,NTT,FINANCE


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-123.94,L-SC,100.00,51821.0,-61728.0,101782.0,4.98,-54.36,196.41,35.27,NaN,-0.61,0.47,26.83,XR,NTT,HOTELS
2,AAVAS,1815.63,2328.00,-30.69,M-SC,98.89,208789.0,14517.0,40317.0,1.73,7.47,19.31,28.22,6.0,0.36,1.91,48.31,XY24,NTT,FINANCE
42,JCHAC,2282.24,3542.00,-21.07,M-SC,97.78,102992.0,-24813.0,95360.0,-0.24,-19.42,92.59,55.20,-57.0,-0.26,0.94,92.76,X40N,BTT,AC
85,UJJIVANSFB,59.50,60.00,-19.23,M-SC,96.67,60724.0,-40426.0,41274.0,0.31,-39.97,67.97,0.84,-27.0,-0.98,0.56,13.83,X40N,NTT,BANKS
20,CAMS,3643.00,5250.99,-6.84,H-SC,95.56,103586.0,1582.0,43444.0,-3.05,1.55,41.94,44.14,261.0,0.04,0.95,32.48,XR,ATH,MISC


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.03
1,25,43.30
2,50,72.11


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    41.85
X40     14.53
X40N    11.56
AR      11.54
XY25     9.06
XR       9.01
SR       1.22
X200     1.21
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    21.78
M-SC    20.14
H-SC    18.11
M-MC     7.93
M-LC     6.83
X-MC     6.51
H-MC     6.38
H-LC     5.35
X-SC     2.75
L-SC     2.46
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,14.60,-15.98,69.22
FINANCE,10.72,-23.61,74.43
FMCG,10.30,-8.08,40.84
IT,7.58,-34.40,101.37
MISC,7.24,-27.61,71.92
INSURANCE,6.23,-0.65,47.97
PAINTS,5.96,-26.15,63.49
CHEMICALS,3.90,-26.49,109.84
ELECTRICAL,3.51,-11.15,57.58


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3296433.0,31
XR,1249904.0,14
X40N,1202582.0,15
AR,984002.0,11
X40,660084.0,10
XY25,216361.0,6
SR,211471.0,2
X200,85674.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2276310.0,18
M-SC,2075293.0,22
X-LC,891062.0,14
M-MC,677068.0,7
X-SC,376708.0,4
H-MC,357044.0,5
L-SC,328169.0,4
X-MC,308109.0,5
M-LC,293438.0,5


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
M-SC       XY24      985743.0      9
H-SC       XY24      963187.0      6
           X40N      571273.0      6
M-MC       XY24      491482.0      4
M-SC       XR        458834.0      5
X-LC       X40       434186.0      6
H-SC       AR        392832.0      2
M-SC       X40N      389962.0      5
H-SC       XR        349018.0      4
H-MC       XY24      294127.0      4
M-LC       XY24      262321.0      4
X-LC       AR        174217.0      3
L-SC       XR        172584.0      2
X-LC       XY25      136137.0      3
X-SC       XR        128879.0      1
           SR        127322.0      1
X-MC       X40N      120840.0      2
X-SC       X40N      120507.0      2
H-LC       AR         94555.0      2
M-SC       AR         88398.0      1
H-LC       X40        88172.0      2
L-SC       XY24       87699.0      1
M-MC       AR         87056.0      1
X-LC       X200       85674.0      1
M-SC       SR         84149.0      1
X-MC       AR         79058.0      1
M-MC       XR         77672.0      1
X-MC       X40        69519.0      1
M-SC       X40        68207.0      1
L-SC       AR         67886.0      1
H-MC       XR         62917.0      1
X-LC       XY24       60848.0      1
X-MC       XY24       38692.0      1
M-LC       XY25       31117.0      1
H-LC       XY25       28249.0      1
M-MC       XY25       20858.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
